In [8]:
import os
import openpyxl
import locale
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
from sqlalchemy import create_engine, exc

load_dotenv()

path_dados = os.getenv('DUSNEI_DATA_DIRECTORY_RELATORIO_OUTROS')

In [9]:
def get_db_engine():
    try:
        db_url = os.getenv('DUSNEI_URL')
        engine = create_engine(db_url)
        # Test connection
        with engine.connect() as connection:
            print('Conexão estabelecida!')
            pass
        print('Banco de dados conectado!')
        return engine
    except exc.SQLAlchemyError as e:
        print(f"Error: {e}")
        return None
        

In [22]:
def query_dados(conn):  
    query = (f"""
        (
            SELECT 
                pfin.pfin_datavcto AS datavcto, 
                pfin.pfin_unid_codigo AS unid_cod,
                pfin.pfin_codentidade AS entidade_cod, 
                pfin.pfin_cnpjcpf AS cnpjcpf, 
                pfin.pfin_numerodcto AS nota, 
                pfin.pfin_valor AS valor,
                pfin.pfin_pger_conta::text AS pger_conta,
                pger.pger_descricao AS descricao, 
                clie.clie_nome AS clie_nome, 
                clie.clie_razaosocial AS razaosocial,
                clie.clie_foneres AS fone_res,
                clie.clie_fonecel AS fone_cel,
                vend.vend_codigo AS vend_cod,
                vend.vend_nome AS vend_nome, 
                vend.vend_supe_codigo AS supe_cod,
                CONCAT(vend.vend_extra16, vend.vend_extra15) AS vend_cel,
                supe.supe_nome AS supe_nome
                FROM pendfin AS pfin
                LEFT JOIN planoger AS pger ON pfin.pfin_pger_conta = pger.pger_conta
                LEFT JOIN clientes AS clie ON pfin.pfin_codentidade = clie.clie_codigo
                LEFT JOIN vendedores AS vend ON clie.clie_vend_codigo = vend.vend_codigo
                LEFT JOIN supervisores AS supe ON vend.vend_supe_codigo = supe.supe_codigo
                WHERE pfin.pfin_status = 'P'
                AND pfin.pfin_datavcto < CURRENT_DATE
                AND pfin.pfin_pger_conta IN ('112131','112140','112201','112202','112203','112521')
                ORDER BY pfin.pfin_datavcto DESC
        )  
    """)
    
    df = pd.read_sql_query(query, conn)
       
    return df

In [23]:
def gerar_excel(df, ws):
    ws.title = f"cobranca"
  
    columns_title = [
        'Index','CodClie','Cliente','RazaoSocial','CNPJ/CPF','Nota','Valor','Tipo','Telefone','Celular','Unid','CodVend','Vendedor','VendCel','CodSupe','Supervisor','Horario','Envio'
    ]
    
    for col_num, column_title in enumerate(columns_title, 1):
        ws.cell(row=1, column=col_num).value = column_title

    for index, row in enumerate(df.itertuples(), start=2):
        horario = '-'
        envio = '-'
        
        tipo = 'Boleto' if row.pger_conta in ['112131','112140','112201'] else 'Cheque'
        
        ws.cell(row=index, column=1).value = row.Index
        ws.cell(row=index, column=2).value = row.entidade_cod
        ws.cell(row=index, column=3).value = row.clie_nome
        ws.cell(row=index, column=4).value = row.razaosocial
        ws.cell(row=index, column=5).value = row.cnpjcpf
        ws.cell(row=index, column=6).value = row.nota
        ws.cell(row=index, column=7).value = row.valor
        ws.cell(row=index, column=7).number_format = 'R$ #,##0.00'
        ws.cell(row=index, column=8).value = tipo
        ws.cell(row=index, column=9).value = row.fone_res
        ws.cell(row=index, column=10).value = row.fone_cel
        ws.cell(row=index, column=11).value = row.unid_cod
        ws.cell(row=index, column=12).value = row.vend_cod
        ws.cell(row=index, column=13).value = row.vend_nome
        ws.cell(row=index, column=14).value = row.vend_cel
        ws.cell(row=index, column=15).value = row.supe_cod
        ws.cell(row=index, column=16).value = row.supe_nome
        ws.cell(row=index, column=17).value = horario
        ws.cell(row=index, column=18).value = envio

In [24]:
def teste_loop():
    conn = get_db_engine()  
    locale.setlocale(locale.LC_MONETARY, 'pt-BR.UTF-8')

    df = query_dados(conn)
    wb = openpyxl.Workbook()
    ws = wb.create_sheet(title="cobranca")
    
    if 'Sheet' in wb.sheetnames:
        wb.remove(wb['Sheet'])
    
    gerar_excel(df, ws)
    
    data_pasta = datetime.now().strftime("%Y-%m-%d")
    nome_arquivo = f'cobranca-{data_pasta}'
    diretorio = os.path.join(path_dados, data_pasta, 'cobranca')

    if not os.path.exists(diretorio):
        os.makedirs(diretorio, exist_ok=True)

    local_arquivo = os.path.join(diretorio, f'{nome_arquivo}.xlsx')
    wb.save(local_arquivo)

teste_loop()

Conexão estabelecida!
Banco de dados conectado!
